# Oppsplitting av kunnskapsartikler til dataprodukt

Notebook for å kjøre samme oppsplitting av artiklene i NKS Kunnskapsbasen som `nks_vdb`, og dytte resultatet til en tabell i BigQuery.

## Setup

In [ ]:
from datetime import datetime
from typing import Iterable

import pandas as pd
from google.cloud import bigquery
from langchain_core.documents import Document
from nks_vdb.knowledgebase import clean_documents, get_column_metadata, split_documents
from nks_vdb.settings import settings

Vi ønsker litt andre metadata- og content-kolonner her enn det som brukes i `nks_vdb`-lasten. 
Vi definerer derfor koden for dokumentinnlastingen på nytt her framfor å hente fra `nks_vdb.knowledgebase`.

(Vi henter bl.a. artikkeltekstene som finnes på nynorsk og engelsk, ikke bare bokmål)


In [ ]:
METADATA_COLUMNS: list[str] = [
    "ArticleType",
    "DataCategories",
    "KnowledgeArticleId",
    "KnowledgeArticle_QuartoUrl",
    "LastModifiedDate",
    "Title",
    "VersionNumber",
]
CONTENT_COLUMNS: list[str] = [
    "Article__c",
    "NKS_User__c",
    "WhoDoesWhat__c",
    "EmployerInformation__c",
    "EmployerInformationInternal__c",
    "InternationalInformation__c",
    "InternationalInformationInternal__c",
    "AdvisorInformation__c",
    "AdvisorInformationInternal__c",
    "How_you_send_a_task__c",
    "NKS_English__c",
    "NKS_English_Employer__c",
    "NKS_Nynorsk__c",
    "NKS_Nynorsk_Employer__c",
]


def __format_query(
    content_column: str, min_length: int = 30, last_modified: datetime | None = None
) -> str:
    """Metode for å formatere spørring til kunnskapsbasen for å hente dokumenter.

    Args:
        content_column:
            Kolonnen som skal benyttes som innhold for dokumentet
        min_length:
            Det må minst være `min_length` antall tegn i innholdet for at det
            skal regnes som et dokument. Dette brukes for å filtrere ut kolonner
            som ikke er NULL eller bare inneholder ' ', men som fortsatt ikke
            inneholder meningsfylt tekst.
        last_modified:
            Valgbar tidspunkt for å filtrere kolonner som er eldre enn
            `last_modified`
    Returns:
        SQL spørring for å hente ut `content_column` som et dokument
    """
    metadata_columns = ", ".join(METADATA_COLUMNS)
    sql = (
        "SELECT {metadata_columns},"
        " {content_column} AS Content,"
        " FROM `kunnskapsbase.kunnskapsartikler`"
        " WHERE PublishStatus = 'Online'"
        " AND {content_column} IS NOT NULL"
        " AND {content_column} != ''"
        " AND CHAR_LENGTH({content_column}) >= {min_length}"
    )
    if last_modified:
        sql += f" AND LastModifiedBQ > '{last_modified.isoformat()}'"
    return sql.format(
        metadata_columns=metadata_columns,
        content_column=content_column,
        min_length=min_length,
    )


def load(last_modified: datetime | None = None) -> Iterable[Document]:
    """Last inn kunnskapsbasen fra BigQuery og produser LangChain dokumenter.

    Args:
        last_modified (valgbar):
            Bare last inn dokumenter nyere enn `last_modified`
    Returns:
        Generator som produserer dokumenter
    """
    from google.cloud import bigquery

    client = bigquery.Client(project=settings.gcp.prosjekt)
    # Vi itererer gjennom alle innholdkolonnene for å minimere minnebruk ved å
    # ikke hente ut alle kunnskapsartikler på en gang
    for column in CONTENT_COLUMNS:
        query = __format_query(column, last_modified=last_modified)
        raw_results = client.query(query).result()
        # For hver rad (mao. hver artikkel) henter vi ut innhold og metadata som
        # tilsammen produserer et dokument
        for row in raw_results:
            metadata = {k: v for k, v in row.items() if k in METADATA_COLUMNS}
            metadata |= get_column_metadata(column, row["ArticleType"], row["Title"])
            content = row["Content"]
            yield Document(page_content=content, metadata=metadata)

## Hent artikkelseksjoner og splitt dem opp

In [ ]:
docs = list(load())

In [ ]:
splits = split_documents(clean_documents(docs), chunk_size=1500, overlap=100)

## Dytte data til BigQuery

`nks_vdb`-koden jobber med dokumentene som Lanchain-dokumenter. For å lettere gjøre skrive dataene til BigQuery kan vi konvertere til en pandas dataframe først.

In [ ]:
# Endrer fra langchain Documents til en pandas DataFrame
data = []
for doc in splits:
    doc_data = {"page_content": doc.page_content}
    doc_data.update(doc.metadata)
    data.append(doc_data)
df_transformed = pd.DataFrame(data)

# Renamer en del kolonner
df_transformed = df_transformed.rename(
    columns={
        "page_content": "content",
        "ArticleType": "type",
        "VersionNumber": "version_number",
        "DataCategories": "data_categories",
        "KnowledgeArticleId": "knowledge_article_id",
        "KnowledgeArticle_QuartoUrl": "url",
        "LastModifiedDate": "last_modified",
        "Title": "title",
        "Section": "section",
        "Tab": "tab",
        "Fragment": "anchor",
        "Headers": "headers",
    }
)

# Stokker om på rekkefølgen
df_transformed = df_transformed[
    [
        "knowledge_article_id",
        "version_number",
        "type",
        "data_categories",
        "last_modified",
        "title",
        "section",
        "tab",
        "headers",
        "content",
        "url",
        "anchor",
    ]
]

# Gjør om `data_categories` og `headers` til en liste av strings slik at de kan være REPEATED fields i BigQuery
df_transformed["data_categories"] = df_transformed["data_categories"].apply(
    lambda x: x.split(",")
)
df_transformed["headers"] = df_transformed["headers"].apply(
    lambda x: [f"{k} {v}" for k, v in x.items()]
)

# Legg på info om hvilken dag lasten skjedde (dagens dato)
df_transformed["load_to_bq_date"] = pd.to_datetime("today").normalize()

In [ ]:
# BigQuery config
prosjekt = settings.gcp.prosjekt
datasett = "kunnskapsbase"
tabellnavn = "kunnskapsartikler_oppsplittet"
table_id = f"{prosjekt}.{datasett}.{tabellnavn}"

schema = [
    bigquery.SchemaField(
        "knowledge_article_id", "STRING", description="Id for artikkelen i Salesforce"
    ),
    bigquery.SchemaField("version_number", "STRING", description="Versjonsnummer"),
    bigquery.SchemaField("type", "STRING", description="Artikkeltype"),
    bigquery.SchemaField(
        "data_categories",
        "STRING",
        mode="REPEATED",
        description="Tema-tagger tilknyttet artikkelen",
    ),
    bigquery.SchemaField(
        "last_modified", "DATETIME", description="Når ble artikkelen sist endret"
    ),
    bigquery.SchemaField("title", "STRING", description="Tittel på artikkelen"),
    bigquery.SchemaField(
        "section",
        "STRING",
        description="Refererer til hvilken artikkel-seksjon i Salesforce innholdet er tilknyttet",
    ),
    bigquery.SchemaField(
        "tab",
        "STRING",
        description="Refererer til hvilken artikkel-tab i Salesforce innholdet er tilknyttet",
    ),
    bigquery.SchemaField(
        "headers",
        "STRING",
        mode="REPEATED",
        description="Underoverskrifter tilknyttet innholdet",
    ),
    bigquery.SchemaField("content", "STRING", description="Selve innholdet"),
    bigquery.SchemaField(
        "url", "STRING", description="Lenke til artikkelvisningen på datamarkedsplassen"
    ),
    bigquery.SchemaField(
        "anchor",
        "STRING",
        description="Anker for innhold i visningen på datamarkedsplassen",
    ),
    bigquery.SchemaField(
        "load_to_bq_date",
        "DATE",
        description="Når dataene ble kopiert til denne tabellen",
    ),
]

client = bigquery.Client(f"{prosjekt}")


def dytt_data_til_bigquery(df, table_id):
    """Hjelpemetode for å skrive pandas dataframe til BigQuery."""
    job = client.load_table_from_dataframe(
        df, table_id, job_config=job_config
    )  # Make an API request.
    job.result()  # Wait for the job to complete.
    print(f"  {len(df)} rader skrevet til tabell")


job_config = bigquery.LoadJobConfig(
    schema=schema,
    write_disposition="WRITE_TRUNCATE",
)

In [ ]:
# Skriv dataene til BigQuery
dytt_data_til_bigquery(df_transformed, table_id)